### Examine Diseases Diagnosed and Missed by O1 and Exomiser using McNemar Test on each disease in Phenopacket-store (and its ancestors)

###### We consider each disease in the Phenopacket-store and its subClassOf ancestors in Mondo to be a pass/fail test for each method. We can then construct a contingency table for each term and use a McNemar test to determine if the methods are significantly different in their ability to diagnose any disease or ancestor in the cohort. 

In [14]:
!pip install statsmodels pandas

In [15]:
# Maximum rank to be considered correct
max_rank = 10

# Limit for testing
limit = 10 # _000_000_000

In [16]:
from typing import List
from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A, PART_OF

adapter = get_adapter("sqlite:obo:mondo") 

# get descendants of hereditary disease
hereditary_disease = "MONDO:0003847"
hereditary_disease_descendants = set([t for t in list(set(adapter.descendants(hereditary_disease, predicates=[IS_A], reflexive=True))) if t.startswith("MONDO")])

# Function to get ancestors of a disease term
def get_ancestors(term_id, prefixes=["MONDO"], restrict_to_hereditary=False):
    """
    Get ancestors of a disease term.
    :param term_id: 
    :param prefixes: 
    :param restrict_to_hereditary: should we only consider hereditary diseases? (MONDO:0003847)
    :return: 
    """
    ancs = [t for t in list(set(adapter.ancestors(term_id, predicates=[IS_A], reflexive=True))) if t.startswith(tuple(prefixes))]
    if restrict_to_hereditary:
        ancs = list(set(ancs).intersection(hereditary_disease_descendants).union([term_id]))
    return ancs 
    
assert(set(get_ancestors("MONDO:0005015", restrict_to_hereditary=False)) ==  \
       set(['MONDO:0000001', 'MONDO:0005015', 'MONDO:0005151', 'MONDO:0001933', 'MONDO:0700096', 'MONDO:0002356', 'MONDO:0004335', 'MONDO:0002908', 'MONDO:0005066']))
assert(set(list(get_ancestors("MONDO:0005015", restrict_to_hereditary=True))) == set(['MONDO:0005015']))

# Function to get the Mondo mappings for an OMIM term
def mondo_mappings_for_omim(term: str, adapter,
                            subject_source="MONDO",
                            predictates=['oio:hasDbXref', "skos:exactMatch"]) -> List[str]: 
    """
    Get the Mondo mappings for an OMIM term.

    Returns:
        str: The mappings.
    """   
    mondo = []
    for m in adapter.sssom_mappings([term]):
        if m.predicate_id in predictates and m.subject_source in subject_source:
            mondo.append(m.subject_id)
    return list(set(mondo))

assert(mondo_mappings_for_omim(term="OMIM:222100", adapter=adapter) == ["MONDO:0005147"])
assert(mondo_mappings_for_omim(term="OMIM:154700", adapter=adapter) == ["MONDO:0007947"])

In [17]:
# okay we need to go to considerable lengths to get the phenopacket file for each Exomiser result file

# for each phenopacket file, find the corresponding exomiser results file
# make a dict of phenopacket_id -> exomiser_results_file

import os
import glob
from tqdm import tqdm
from warnings import warn

# Define the directories
phenopacket_dir = "../supplemental_data/phenopackets/"
exomiser_results_dir = "../outputdir_all_2024_07_04/pheval_disease_results/"

# Get all result files
result_files = glob.glob(os.path.join(exomiser_results_dir, "*-pheval_disease_result.tsv"))

# Initialize the dictionary
phenopacket2results = {}

# Process each result file
for result_file in tqdm(result_files):
    # Get the base name of the result file (without extension and suffix)
    base_name = os.path.basename(result_file).replace("-pheval_disease_result.tsv", "")
    
    # Construct the pattern for matching prompt files
    prompt_pattern = os.path.join(phenopacket_dir, f"{base_name}.json")
    
    # Find matching phenopacket files
    matching_phenopackets = glob.glob(prompt_pattern)
    
    # Check if there's exactly one matching phenopacket file
    if len(matching_phenopackets) == 1:
        phenopacket2results[matching_phenopackets[0]] = result_file
    elif len(matching_phenopackets) == 0:
        warn(f"No matching phenopacket_dir file found for {result_file}")
    else:
        warn(f"Multiple matching phenopacket_dir files found for {result_file}")

print(f"Total mappings: {len(phenopacket2results)}")

phenopacket2results

100%|██████████| 5212/5212 [00:00<00:00, 36705.28it/s]


Total mappings: 5212


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from google.protobuf.json_format import Parse
from phenopackets import Phenopacket
from statsmodels.stats.contingency_tables import mcnemar
from tqdm import tqdm
import os
import glob
import csv
from warnings import warn

# Initialize dictionaries to store pass/fail outcomes for each method (as lists of outcomes)
exomiser_mondo_terms_pass_fail = {}
o1_mondo_terms_pass_fail = {}

# Function to safely append results to the ancestor dictionary
def append_result(results_dict, ancestor, is_correct):
    if ancestor not in results_dict:
        results_dict[ancestor] = []
    results_dict[ancestor].append(is_correct)

# Step 1: Exomiser loop - Iterate over the phenopacket files and calculate Exomiser results
for i, (phenopacket_file, exomiser_results_file) in tqdm(enumerate(phenopacket2results.items()),
                                                         total=len(phenopacket2results),
                                                         desc="Processing phenopackets for Exomiser"):
    # Parsing phenopackets from json
    with open(phenopacket_file, 'r') as jsfile:
        phenopacket = Parse(message=Phenopacket(), text=jsfile.read())
        if len(phenopacket.diseases) > 1:
            raise Exception(f"Multiple diseases found in {phenopacket_file}")
        elif len(phenopacket.diseases) == 0:
            raise Exception(f"No diseases found in {phenopacket_file}")
        
        # Get the disease ID (OMIM ID)
        omim_id = phenopacket.diseases[0].term.id 
        omim_name = phenopacket.diseases[0].term.label
        
        # Get Mondo mappings for the disease
        mondo_terms = mondo_mappings_for_omim(omim_id, adapter)
        if len(mondo_terms) == 0:
            raise Exception(f"No Mondo mappings found for {omim_id}")
        elif len(mondo_terms) > 1:
            raise Exception(f"Multiple Mondo mappings found for {omim_id}")
        mondo_term = mondo_terms[0]
        mondo_term_and_ancestors = get_ancestors(mondo_term)
        
        # Step 2: Check Exomiser results for the correct diagnosis
        results = pd.read_csv(exomiser_results_file, sep="\t")
        filtered_results = results[(results["rank"] <= max_rank) & (results["is_correct"] == True)]
        
        # Reality check - correct_ID should match omim_id
        if len(filtered_results['correct_ID'].unique()) > 1:
            assert(list(filtered_results['correct_ID'].unique())[0] == omim_id)
        
        # Check if the correct diagnosis is in the top 'max_rank' results
        correct_in_top = len(filtered_results) > 0
        
        # Add pass/fail for Exomiser method to each ancestor (store as a list of outcomes)
        for ancestor in mondo_term_and_ancestors:
            append_result(exomiser_mondo_terms_pass_fail, ancestor, correct_in_top)

    if i >= limit:
        break

# Step 2: o1 loop - Similar logic for o1 method
# Directories for the results
o1_dir = "../outputdir_all_2024_07_04/gpt_o1_disease_results/"

# Load the correct answers TSV file into a dictionary
correct_answers_dict = {}
correct_answers_path = '../supplemental_data/correct_results.tsv'

# Read the correct answers file
with open(correct_answers_path, 'r', newline='') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        correct_disease_name = row[0]
        correct_ID = row[1]
        prompt_file_name = row[2]
        correct_answers_dict[prompt_file_name] = (correct_ID, correct_disease_name)

# Loop through correct answers and process o1 results
for i, (prompt_file_name, correct_info) in tqdm(enumerate(correct_answers_dict.items()), total=len(correct_answers_dict), desc="Processing correct answers for o1"):
    # Find o1 result file
    o1_results_file = glob.glob(os.path.join(o1_dir, prompt_file_name + '.tsv'))

    # Ensure there's exactly one match
    if len(o1_results_file) != 1:
        raise Exception(f"Expected 1 o1 result file for {prompt_file_name}, but found {len(o1_results_file)}.")
    else:
        o1_results_file = o1_results_file[0]

    omim_id = correct_answers_dict[prompt_file_name][0]
    omim_label = correct_answers_dict[prompt_file_name][1]
        
    # Get Mondo mappings for the disease
    mondo_terms = mondo_mappings_for_omim(omim_id, adapter)
    if len(mondo_terms) == 0:
        raise Exception(f"No Mondo mappings found for {omim_id}")
    elif len(mondo_terms) > 1:
        raise Exception(f"Multiple Mondo mappings found for {omim_id}")
    mondo_term = mondo_terms[0]
    mondo_term_and_ancestors = get_ancestors(mondo_term)
        
    # Step 4: Check o1 results
    results = pd.read_csv(o1_results_file, sep="\t")
    filtered_results = results[(results["rank"] <= max_rank) & (results["is_correct"] == True)]
        
    # Reality check - correct_ID should match omim_id
    if len(filtered_results['correct_ID'].unique()) > 1:
        assert(list(filtered_results['correct_ID'].unique())[0] == omim_id)
    
    # Check if the correct diagnosis is in the top 'max_rank' results
    correct_in_top = len(filtered_results) > 0
    
    # Add pass/fail for o1 method to each ancestor (store as a list of outcomes)
    for ancestor in mondo_term_and_ancestors:
        append_result(o1_mondo_terms_pass_fail, ancestor, correct_in_top)

    if i >= limit:
        break

In [ ]:
# Step 3: Get the union of Mondo terms across Exomiser and o1
all_mondo_terms = set(exomiser_mondo_terms_pass_fail.keys()).intersection(set(o1_mondo_terms_pass_fail.keys()))

# Step 4: Perform McNemar's test for each Mondo term
mcnemar_results = {}

for mondo_term in all_mondo_terms:
    # Collect pass/fail outcomes for Exomiser and o1 for this Mondo term
    exomiser_outcomes = exomiser_mondo_terms_pass_fail.get(mondo_term, [])
    o1_outcomes = o1_mondo_terms_pass_fail.get(mondo_term, [])

    # # Ensure both lists are of the same length
    if len(exomiser_outcomes) != len(o1_outcomes):
        warn(f"Length mismatch for {mondo_term}: Exomiser outcomes: {len(exomiser_outcomes)}, o1 outcomes: {len(o1_outcomes)}")
        continue
    
    # Create the contingency table for this Mondo term
    contingency_table = pd.crosstab(pd.Series(o1_outcomes), pd.Series(exomiser_outcomes))

    # Perform McNemar test
    result = mcnemar(contingency_table)

    # Store the result
    mcnemar_results[mondo_term] = result.pvalue

# Step 5: Display the McNemar's test results
for mondo_term, pvalue in mcnemar_results.items():
    # get the label for the mondo term
    label = adapter.label(mondo_term)
    print(f"Mondo Term: {mondo_term} {label}, McNemar p-value: {pvalue}")